## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Completion API, RAG и Function Calling.

Для начала, установим OpenAI SDK и другие полезные библиотеки:

In [26]:
%pip install --upgrade openai pydantic dotenv tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.1 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё полезная функция на будущее для печати markdown:

In [2]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

## Авторизация

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Это можно сделать несколькими способами:

* через iam-токен. Для получения iam-токена необходимо создать авторизованный ключ доступа к сервисному аккаунту, и знать `service_accound_id`, `key_id` и `private_key`. Скачайте файл с ключами доступа `authorized_key.json`, и добавьте к нему поле `folder_id`
* **[рекомендованный способ]** через ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere или в переменных окружения. Если вы используете набор переменных окружения в файле `.env` - запустите следующую ячейку для их загрузки.

In [1]:
from dotenv import load_dotenv
load_dotenv()

False

In [3]:
import json
import os
from util.iam_auth import get_iam

if os.path.exists('authorized_key.json'):
    with open('authorized_key.json') as f:
        auth_key = json.load(f)
    api_key = get_iam(auth_key['service_account_id'],auth_key['id'],auth_key['private_key'])
    folder_id = auth_key['folder_id']
    print(f"Using IAM Token Auth with folder_id={folder_id}")
else:
    folder_id = None

Using IAM Token Auth with folder_id=b1gbicod0scglhd49qs0


Если файл с ключем не найден, то используем значения `api_key` из переменных окружения:

In [10]:
if folder_id is None:
    folder_id = os.environ["folder_id"]
    api_key = os.environ["api_key"]

Создадим какую-нибудь модель из Foundation Models и убедимся, что она кое-что знает про вина. 

> ВНИМАНИЕ: Для правильной работы необходимо передать `folder_id` в параметр `project` при создании объекта OpenAI SDK.

In [4]:
import os
from openai import OpenAI

model = f"gpt://{folder_id}/yandexgpt/rc"
model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
model = f"gpt://{folder_id}/gpt-oss-120b/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

In [14]:
res = client.responses.create(
    model = model,
    input = "Какое вино можно пить со стейком?"
)

printx(res.output_text)

Сочетание вина со стейком — это классика, и выбор зависит от типа мяса, способа приготовления и личных предпочтений. Вот основные рекомендации:

### 1. **Красные вина — лучший выбор**
Стейк (особенно из говядины) имеет насыщенный вкус и жирность, поэтому хорошо сочетается с полновкусными красными винами, которые обладают достаточной структурой и танинами.

---

### 🔥 Популярные варианты:

#### 🍷 **Каберне Совиньон**
- **Почему подходит:** Высокие танины и кислотность «очищают» жир с мяса, подчёркивая его вкус.
- **Ноты:** чёрная смородина, перец, дуб, мята, табак.
- **Идеален для:** прожаренного стейка (рибай, филе миньон, томагавк).

#### 🍷 **Малбек**
- **Почему подходит:** Плотный, фруктовый, с мягкими танинами — отлично сбалансирует жирность.
- **Ноты:** чёрная вишня, слива, шоколад, дым.
- **Идеален для:** стейков с дымным привкусом (гриль, BBQ), особенно из аргентинского Мендосы.

#### 🍷 **Сира (Шираз)**
- **Почему подходит:** Нафталиновая пряность и плотная текстура хорошо сочетаются с обугленной корочкой.
- **Ноты:** чёрные ягоды, перец, дым, специи.
- **Идеален для:** стейков на углях, с пряными маринадами.

#### 🍷 **Бордо (сбор на основе Каберне и Мерло)**
- **Почему подходит:** Сложный, структурированный, с балансом танинов и фруктов.
- **Идеален для:** богатых, сочных стейков — особенно рибая или сэрлоина.

#### 🍷 **Неббиоло (например, Бароло)**
- **Почему подходит:** Очень танинный и ароматный — для ценителей.
- **Ноты:** вишня, роза, гриб, кожа.
- **Идеален для:** выдержанного стейка, особенно с трюфельным маслом.

---

### ❌ Что лучше избегать:
- **Лёгкие красные вина** (Пино Нуар, Божоле) — могут «потеряться» на фоне стейка.
- **Белые вина** — редко сочетаются, **кроме исключений**:
  - **Выдержанное шардоне** (полнотелое, с дубом) может подойти к стейку с грибным соусом.
  - **Орвието или Совиньон Блан** — только если стейк лёгкий (например, телятина) и с цитрусовым маринадом.

---

### 💡 Совет:
Если стейк с **грибным соусом** — выбирай вина с землистыми нотами (Пино Нуар из Бургундии, Бароло).  
Если с **острым соусом** — Малбек или Шираз сбалансируют жгучесть.

---

### Примеры по стейкам:
| Стейк          | Рекомендуемое вино               |
|----------------|----------------------------------|
| Рибай          | Каберне Совиньон, Бордо          |
| Филе миньон    | Пино Нуар (если нежирный), Каберне |
| Томагавк       | Малбек, Шираз, Каберне           |
| Стрейк (стейк) | Сира, Каберне Совиньон           |

---

**Итог:** Лучший выбор — **Каберне Совиньон** или **Малбек**. Они универсальны, доступны и идеально подходят к сочному стейку.

## Responses API

Когда мы используем запрос вида `client.responses.create` - мы используем так называемый Responses API. Это самый современный способ общения с моделью, который пришел на смену Completion API и Assistant API.

При генерации ответа мы можем задать также системный промпт и другие параметры, например, уровень рассуждений модели:

In [15]:
res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    instructions = "Ты - опытный сомелье, задача которого - консультировать пользователя в вопросах выбора вина.",
    input = "Привет! Какое вино посоветуете?"
)

printx(res.output_text)

Здравствуйте! С удовольствием помогу подобрать вино — но для начала давайте немного уточним детали, чтобы выбрать действительно хорошее сочетание под ваш вкус и ситуацию. Ответьте, пожалуйста, на несколько коротких вопросов:

1. **Какой тип вина вы предпочитаете?**  
   — Белое, красное, розовое, игристое или десертное?

2. **Какой вкус вам ближе?**  
   — Сухое, полусладкое, сладкое?  
   — Лёгкое и свежее или насыщенное, терпкое?

3. **На какое блюдо или случай выбираем вино?**  
   — Например: аперитив, морепродукты, мясо, сыр, десерт, просто попить с друзьями.

4. **Есть ли предпочтения по стране происхождения?**  
   — Франция, Италия, Испания, Германия, Новая Зеландия и т.д.

5. **Ориентируетесь на ценовой диапазон?**  
   — Бюджетное, средний сегмент или что-то премиальное?

Чем больше деталей — тем точнее будет рекомендация! 🍷

Чтобы продолжить диалог, мы можем либо передать модели на вход всю историю диалога, либо указать ID предыдущего ответа, начиная с которого нужно продолжить диалог (при этом в предыдущем диалоге нам нужно указать `store=True`, чтобы переписка сохранялась):

In [16]:
print(f"ID предыдущего ответа: {res.id}")

res = client.responses.create(
    model = model,
    reasoning = { "effort" : "low" },
    store = True,
    previous_response_id = res.id,
    input = "Я буду есть стейк!"
)

printx(res.output_text)

ID предыдущего ответа: 5f509178-6df9-45f1-bdcc-cc4a55181cfa


Отличный выбор — стейк и вино: это классика! 🥩🍷

Для стейка лучше всего подойдёт **насыщенное, структурированное красное вино** с хорошей танинностью и глубоким вкусом — оно отлично сбалансирует жирность мяса и подчеркнёт его умами.

Вот несколько отличных вариантов на выбор:

---

### 🔹 **Каберне Совиньон**  
— **Почему подходит:** плотный, с танинами, чёрными ягодами (чёрная смородина, чёрная вишня), нотками дуба и лакрицы.  
— **Идеально:** к прожаренному стейку (особенно рибай или вагю).  
— **Где искать:** Направления — *Бордо* (Франция), *Напа Вэлли* (Калифорния), *Чили* (Колчагуа).  
— **Пример:** Château Lanessan (Бордо), Caymus Vineyards (Калифорния).

---

### 🔹 **Сира (Шираз)**  
— **Почему подходит:** насыщенный, с сочными тонами перца, дыма, тушёных ягод и шоколада.  
— **Идеально:** к стейку с дымным привкусом (гриль, BBQ).  
— **Где искать:** *Кот-Роти* (Франция), *Баросса Вэлли* (Австралия).  
— **Пример:** Yalumba "The Signature" (Австралия), E. Guigal Crozes-Hermitage.

---

### 🔹 **Мальбек**  
— **Почему подходит:** сочный, фруктовый, с шелковистыми танинами и нотами чёрной сливы, ванили, кофе.  
— **Идеально:** к аргентинскому стейку или просто хорошо прожаренному мясу.  
— **Где искать:** *Мендоса* (Аргентина) — мировой эталон.  
— **Пример:** Catena Zapata "Alta" Malbec, Achaval-Ferrer Malbec.

---

### 🔹 **Премиум-вариант: Бароло или Брунелло ди Монтальчино**  
— Если хотите что-то особенное:  
— **Бароло** (Италия, из сорта Неббиоло) — мощное, ароматное, с розой, вишней, трюфелем.  
— **Брунелло** (Тоскана) — концентрированное, выдерживается, с телом и долгим послевкусием.

---

### ✅ Простой совет:
Если стейк с соусом (например, пеперсей, грибной или красное вино), выбирайте **ещё более насыщенное вино** — чтобы оно не «потерялось».

---

Хотите, я подберу конкретную бутылку под ваш бюджет и доступность (например, в магазине вашего города или онлайн)? Назовите примерный ценовой диапазон — и будет точечный совет! 🍷

## Function Calling

Предположим, мы делаем ассистента для магазина вин, который торгует винами по некоторому прайс-листу. Возьмём пример такого прайс-листа:

In [5]:
import pandas as pd

pl = pd.read_excel("data/wine-price-ru.xlsx")
pl

,Id,Name,Country,Price,WHPrice,etc,Acidity,Color,Volume
0,56885,САССИКАЙЯ КР СХ,IT,27799.000,19459.3000,NaN,Сухое,Красное,0.750
1,666560,СИЕПИ МАЗЕЙ КР СХ,IT,15999.000,11199.3000,NaN,Сухое,Красное,0.750
2,533769,ПАЛАФРЕНО КР СХ,IT,14999.004,10499.3028,NaN,Сухое,Красное,0.750
3,93733,АНТ ТИНЬЯНЕЛЛО КР СХ,IT,14499.012,10149.3084,NaN,Сухое,Красное,0.750
4,644863,ШАТО МОНРОЗ КР СХ,FR,12999.000,9099.3000,от промо цены,Сухое,Красное,0.750
...,...,...,...,...,...,...,...,...,...
747,61418,КАГОР ТАМ КР СЛ,RU,179.004,125.3028,от промо цены,Сладкое,Красное,0.700
748,615581,ДЖАСТ МЕРЛО КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
749,615582,ДЖАСТ КБСВ КР СХ,FR,149.004,104.3028,NaN,Сухое,Красное,0.187
750,83302,АДАГУМ КБСВ КР СХ,RU,119.004,83.3028,NaN,Сухое,Красное,0.187


Предположим, мы хотим научиться отвечать на вопросы по прайс-листу, например, какое есть самое дешевое красное вино из Италии. Это можно сделать несколькими способами:

* Попытаться закинуть прайс-лист в контекст модели. Этот подход будет работать только для очень небольших таблиц. Конечно, можно попробовать использовать RAG (подробнее про это ниже), но без видения всей таблицы модель не сможет найти самое дешевое вино.
* Попытаться организовать трансляцию запроса не естественном языке в SQL-подобный язык. Это идеальный вариант, но его сложно сделать без ошибок без fine-tuning-а модели. 
* Извлечь из текстового запроса основные параметры того, что хочет пользователь, и затем сформировать на этой основе запрос, извлечающий данные из таблицы. Такой подход описан, например, в статье [Querying Databases with Function Calling](https://arxiv.org/html/2502.00032v1)

В любом случае, два последних варианта подразумевают использование технологии Function Calling - когда модель может сама решить, когда ей надо вызвать некоторую **внешнюю функцию**, и какие параметры нужно ей передать.

Чтобы function calling работал - нам надо сообщить LLM о доступных **инструментах**. Это можно сделать, передав с помощью JSON-схемы описание возможностей таких инструментов и их параметров.

Часто, чтобы не писать JSON-схему для function calling вручную, используют типизированные объекты Pyton pydantic. Для извлечения параметров запроса о вине, мы создадим такой объект: 

In [6]:
from pydantic import BaseModel, Field
from typing import Optional

class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

Теперь создадим инструмент (tool) и передадим его в запрос модели. Также в инструкции ассистенту пропишем, что он может использовать Function Calling.

In [7]:
tools = [
    {
        "type": "function",
        "name": "Exercise",
        "description": "Вызывай, когда пользователь хочет получить информацию о конкретных винах или их стоимости.",
        "parameters": SearchWinePriceList.model_json_schema(),
    }
]

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин
или цены, то используй Function Calling.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

res = client.responses.create(
    model = model,
    store = True,
    tools = tools,
    instructions = instruction,
    input = "Какое самое дешевое вино из Австралии?"
)
res.to_dict()

{'id': '497b3603-f8e7-48d6-ac5a-49433fe9ecf9',
 'created_at': 1758659664621.0,
 'error': None,
 'incomplete_details': None,
 'instructions': '\nТы - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина\nи рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. \nПосмотри на всю имеющуюся в твоем распоряжении информацию\nи выдай одну или несколько лучших рекомендаций. Если вопрос касается конкретных вин\nили цены, то используй Function Calling.\nЕсли что-то непонятно, то лучше уточни информацию у пользователя.\n',
 'metadata': None,
 'model': 'gpt://b1gbicod0scglhd49qs0/qwen3-235b-a22b-fp8/latest',
 'object': 'response',
 'output': [{'arguments': '{"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"}',
   'call_id': 'chatcmpl-tool-f25854d16755455492e46b1b18becb1c',
   'name': 'Exercise',
   'type': 'function_call',
   'id': 'chatcmpl-tool-f25854d16755455492e46b1b18becb1c',
   'status': 'completed',
  

Видим, что в итоге вернулся ответ, указывающий, что надо сделать Funtion Call для получения данных из таблицы. При это LLM извлекла из текста все параметры запроса.

Реализуем функцию, которая возвращает список вин по параметрам, заданным в виде объекта `SearchWinePriceList`:

In [8]:
country_map = {
    "IT": "Италия",
    "FR": "Франция",
    "ES": "Испания",
    "RU": "Россия",
    "PT": "Португалия",
    "AR": "Армения",
    "CL": "Чили",
    "AU": "Австрия",
    "GE": "Грузия",
    "ZA": "ЮАР",
    "US": "США",
    "NZ": "Новая Зеландия",
    "DE": "Германия",
    "AT": "Австрия",
    "IL": "Израиль",
    "BG": "Болгария",
    "GR": "Греция",
    "AU": "Австралия",
}

revmap = {v.lower(): k for k, v in country_map.items()}


def find_wines(req):
    x = pl.copy()
    if req.country and req.country.lower() in revmap.keys():
        x = x[x["Country"] == revmap[req.country.lower()]]
    if req.acidity:
        x = x[x["Acidity"] == req.acidity.capitalize()]
    if req.color:
        x = x[x["Color"] == req.color.capitalize()]
    if req.name:
        x = x[x["Name"].apply(lambda x: req.name.lower() in x.lower())]
    if req.sort_order and len(x)>0:
        if req.sort_order == "cheapest":
            x = x.sort_values(by="Price")
        elif req.sort_order == "most expensive":
            x = x.sort_values(by="Price", ascending=False)
        else:
            pass
    if x is None or len(x) == 0:
        return "Подходящих вин не найдено"
    return "Вот какие вина были найдены:\n" + "\n".join(
        [
            f"{z['Name']} ({country_map.get(z['Country'],'Неизвестно')}) - {z['Price']}"
            for _, z in x.head(10).iterrows()
        ]
    )


print(find_wines(SearchWinePriceList(country="Австралия", sort_order="cheapest")))

Вот какие вина были найдены:
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


В ответ на Function Call нам нужно сформировать ответ от функции, и передать её обратно для обработки языковой моделью. В нашем случае функция может быть только одна, поэтому мы не проверяем название функции, а всегда запрашиваем поиск в таблице:

In [9]:
import json 

tool_calls = [item for item in res.output if item.type == "function_call"]

if tool_calls:
    out = []
    for call in tool_calls:
        print(f" + Обрабатываем: {call.name} (call_id={call.call_id}, args={call.arguments})")
        try:
            args = json.loads(call.arguments)
            args = SearchWinePriceList.model_validate(args)
            result = find_wines(args)
        except Exception as e:
            result = f"Ошибка: {e}"
        print(f" + Результат: {result}")
        out.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })
        res = client.responses.create(
            model=model,
            input=out,
            tools=tools,
            previous_response_id=res.id,
            store=True
        )

printx(res.output_text)

 + Обрабатываем: Exercise (call_id=chatcmpl-tool-f25854d16755455492e46b1b18becb1c, args={"country": "Австралия", "sort_order": "cheapest", "what_to_return": "wine info"})
 + Результат: Вот какие вина были найдены:
 ДЖИНДАЛИ МЕРЛО КР ПСХ (Австралия) - 499.0
 ДЖИНДАЛИ КБСВ КР ПСХ (Австралия) - 499.0
 ЧОЛК ХИЛЛ ШИРАЗ КР СХ (Австралия) - 509.0
 ПИТ'С ПЮР ШИРАЗ КР ПСХ (Австралия) - 579.0
 ПИТ'С ПЮР ПННР КР ПСХ (Австралия) - 579.0
 СТАМП ДЖАМП КР СХ (Австралия) - 789.0
 ЛИНД БИН50 ШИР КР ПСХ (Австралия) - 899.0
 ЛЭКИ ШИРАЗ КРСХ (Австралия) - 978.996
 СТЭДФАСТ ШИР КАБ КРСХ (Австралия) - 999.0
 ТИРРЕЛЗ ШИР КР СХ (Австралия) - 1098.996


Самое дешёвое вино из Австралии в вашем списке — **ДЖИНДАЛИ МЕРЛО КР ПСХ** и **ДЖИНДАЛИ КБСВ КР ПСХ**, оба по цене **499 рублей**.

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула соответствующий ответ

## Релизуем агента с Function Calling

Для реализации полноценного ассистента добавим ещё функцию добавления вина в корзину, распечатку корзины и передачи общения оператору.

Немного структурируем наш код:
* Добавим функцию для обработки вызова фукции прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов, а также автоматически поддерживать диалог, запоминая идентификаторы предыдущих ответов.

> Чтобы ассистент мог поддерживать диалог сразу с несколькими пользователями, нам нужно также ввести некоторый идентификатор сессии `session_id`, и для каждой сессии помнить свою историю переписки и `id` последнего сообщения. Идентификатор сессии мы также будет передавать во все функции `session_id`.


In [10]:
class SearchWinePriceList(BaseModel):
    """Эта функция позволяет искать вина в прайс-листе по одному или нескольким параметрам."""

    name: str = Field(description="Название вина", default=None)
    country: str = Field(description="Страна на русском языке", default=None)
    acidity: str = Field(
        description="Кислотность (сухое, полусухое, сладкое, полусладкое)", default=None
    )
    color: str = Field(description="Цвет вина (красное, белое, розовое)", default=None)
    sort_order: str = Field(
        description="Порядок выдачи (most expensive, cheapest, random, average)",
        default=None,
    )
    what_to_return: str = Field(
        description="Что вернуть (wine info или price)", default=None
    )

    def process(self, session_id):
        return find_wines(self)

Также в виде функции реализуем функциональность передачи управления оператору. В данном случае мы будем просто устанавливать некоторый глобальный флаг `handover` - это очень грубое решение и сделано для простоты, в реальной жизни надо будет подумать над более правильной логикой, поддерживающей нескольких пользователей.

In [11]:
handover = False

class Handover(BaseModel):
    """Эта функция позволяет передать диалог человеку-оператору поддержки"""

    reason: str = Field(
        description="Причина для вызова оператора", default="не указана"
    )

    def process(self, session_id):
        global handover
        handover = True
        return f"Я побежала вызывать оператора, ваш {session_id=}, причина: {self.reason}"

Также реализуем функцию добавления вин в корзину. Чтобы для каждого пользователя была своя корзина, будем привязывать её к идентификатору сессии:

In [12]:
carts = {}

class AddToCart(BaseModel):
    """Эта функция позволяет положить или добавить вино в корзину"""

    wine_name: str = Field(
        description="Точное название вина, чтобы положить в корзину", default=None
    )
    count: int = Field(
        description="Количество бутылок вина, которое нужно положить в корзину",
        default=1,
    )

    def process(self, session_id):
        if session_id not in carts:
            carts[session_id] = []
        carts[session_id].append(self)
        return f"Вино {self.wine_name} добавлено в корзину, число бутылок: {self.count}"

Наконец, оформим функцию для показа корзины:

In [13]:
class ShowCart(BaseModel):
    """Эта функция позволяет показать содержимое корзины"""

    def process(self, session_id):
        if session_id not in carts or len(carts[session_id]) == 0:
            return "Корзина пуста"
        return "В корзине находятся следующие вина:\n" + "\n".join(
            [f"{x.wine_name}, число бутылок: {x.count}" for x in carts[session_id]]
        )

Теперь реализуем главный класс `Agent`, который будет брать на себя обработку функций. В качестве `tools` будем передавать список описанных нами ранее классов.

In [63]:
class Agent():
    user_sessions = {}

    def __init__(self, instruction, tools = [], session_id='default', model=model,tool_choice='auto'):
        self.instruction = instruction
        self.model = model
        self.tool_choice = tool_choice
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        if session_id not in self.user_sessions:
            self.user_sessions[session_id] = {
                "last_reply_id" : None,
                "history" : [],
            }

    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default'):
        s = self.user_sessions.get(session_id,{ 'last_reply_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            tool_choice = self.tool_choice,
            instructions = self.instruction,
            previous_response_id = s['last_reply_id'],
            input = message
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # MCP Approval Requests
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['last_reply_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id]=s
        return res

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Создадим нашего агента - винного сомелье. В системном промпте пропишем ему возможные варианты для вызова функции, чтобы облегчить задачу.

In [15]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию SearchWinePriceList.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[SearchWinePriceList, Handover, AddToCart, ShowCart],
)

In [16]:
printx(wine_agent("Какое вино пьют со стейком?").output_text)

Со стейком отлично сочетаются насыщенные красные вина с хорошей структурой и танинами. Вот несколько отличных вариантов:

- **Каберне Совиньон** — насыщенное вино с танинами и нотами чёрных ягод, идеально подчёркивает вкус говядины.
- **Мерло** — более мягкое, чем Каберне, с фруктовыми оттенками, подойдёт, если стейк приготовлен менее интенсивно.
- **Сира (Шираз)** — с пряными, дымными нотами, отлично гармонирует с обугленной корочкой стейка.
- **Мальбек** — особенно хорош с сочным стейком, обладает плотным телом и яркой фруктовостью.

Хотите, чтобы я подобрал конкретные вина из нашего ассортимента? Могу поискать, например, хороший Каберне Совиньон или Мальбек.

In [17]:
printx(wine_agent("Какие вина Кьянти есть в продаже?").output_text)

 + Обрабатываем: SearchWinePriceList ({"name": "Кьянти", "what_to_return": "wine info"})


Вот вина **Кьянти** (красные, сухие), которые есть в продаже:

1. **Кверчаб Кьянти** — 2499 ₽  
2. **Полиц Кьянти** — 1749,76 ₽  
3. **Касал Кьянти Супериоре** — 1349 ₽  
4. **Век Кант Кьянти** — 1099 ₽  
5. **Пределла Кьянти** — 999 ₽  
6. **Зонин Кьянти** — 699 ₽  
7. **Пределла Кьянти** — 369 ₽  

Кьянти — классическое итальянское красное вино из региона Тоскана, отлично подходит к мясу, пасте и сырам. Если хотите, могу порекомендовать лучший вариант по соотношению цена-качество или помочь добавить вино в корзину.

In [18]:
printx(wine_agent("Добавь в корзину Полиц Кьянти, три бутылки").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "ПОЛИЦ КЬЯНТИ КР СХ (Италия)", "count": 3})


In [19]:
printx(wine_agent("Ещё положи в корзину Зонин Кьянти").output_text)

 + Обрабатываем: AddToCart ({"wine_name": "ЗОНИН КЬЯНТИ КР СХ (Италия)", "count": 1})


In [20]:
printx(wine_agent("Что у меня в корзине?").output_text)

 + Обрабатываем: ShowCart ({})


В вашей корзине сейчас:

- **Полиц Кьянти** — 3 бутылки  
- **Зонин Кьянти** — 1 бутылка  

Если хотите оформить заказ или добавить ещё что-то — дайте знать! Также могу порекомендовать, какое из этих вин лучше подать к определённому блюду.

In [21]:
printx(wine_agent("Вызови оператора, хочу оформить доставку!").output_text)

 + Обрабатываем: Handover ({"reason": "Оформление доставки"})


Оператор уже в пути — с вами свяжутся, чтобы уточнить детали доставки. Спасибо, что выбрали нас! 🍷  
Если захотите что-то добавить или изменить в заказе — просто скажите.

## Model Context Protocol

Мы научились интегрировать в ассистента локальные функции, но иногда бывает так, что функции предоставляются какими-то внешними поставщиками информации. Предположим, мы хотим сделать универсального ассистента, который сможет анализировать цены в различных интернет-магазинах. Для этого нужно, чтобы интернет-магазины предоставляли доступ к своим прайс-листам в виде удалённых функций, доступных через протокол MCP: Model Context Protocol. 

Реализуем такой MCP-сервер для доступа к прайс-листу магазина. Для этого мы используем библиотеку [FastMCP](https://gofastmcp.com/). Пример реализации сервера доступен [здесь](mcp-server/mcp-wine-shop.py), запустить его можно командой
```bash
fastmcp run mcp-wine-shop.py -t sse -p 3000 --host 0.0.0.0
```

Вы также можете запустить сервер с помощью [FastMCP Cloud](https://fastmcp.cloud/) используя [этот GitHub-репозиторий](https://github.com/yandex-datasphere/advanced-assistant-mcp)

In [22]:
mcp_tool = {
            "type": "mcp",
            "server_label": "Wine-Shop",
            "server_description": "Функция для запроса цен на вино в винном магазине",
            #"server_url": "https://wineparadise.fastmcp.app/mcp",
            "server_url": "http://cathy.ycloud.eazify.net:8000/sse",
            "require_approval": "never",
    }

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина. 
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool],
)

In [24]:
printx(wine_agent("Сколько стоит самый дорогой Мерло?").output_text)

Самое дорогое Мерло в наличии — **Лефкадия Мерло (Россия)**, его цена составляет **2449,66 ₽** за бутылку.  

Это насыщенное красное сухое вино с хорошей структурой и фруктовыми нотами — отличный выбор для ценителей. Хотите добавить его в корзину?

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по винам, однако если мы хотим сделать его знания более предсказуемыми - необходимо дополнить их специализированной информацией из текстовой базы знаний. Для этого мы можем использовать RAG - подход.

В качестве текстовой базы знаний в нашем примере возьмём тексты о винах из энциклопедии: про разные сорта винограда и про регионы произрастания:


In [27]:
from glob import glob
import pandas as pd
import tiktoken

def get_tokenizer(model):
    try:
        return tiktoken.encoding_for_model(model)
    except:
        return tiktoken.encoding_for_model("gpt-5-nano") 

def get_token_count(filename):
    tokenizer = get_tokenizer(model)
    with open(filename, "r", encoding="utf8") as f:
        return len(tokenizer.encode(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in list(glob("data/wines.txt/*.txt"))+list(glob("data/regions.txt/*.txt"))
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/wines.txt/Альбариньо.txt,763,2387
1,data/wines.txt/Блауфранкиш.txt,787,2596
2,data/wines.txt/Вионье.txt,651,2117
3,data/wines.txt/Виура.txt,324,1038
4,data/wines.txt/Гевюрцтраминер.txt,736,2236
...,...,...,...
125,data/regions.txt/Штирия.txt,619,2078
126,data/regions.txt/Элгин.txt,346,1081
127,data/regions.txt/Элим.txt,346,1110
128,data/regions.txt/Эльзас.txt,530,1887


Чтобы агент мог обращаться к этим файлам - загружаем их в облако:

In [28]:
from tqdm.auto import tqdm
tqdm.pandas()

def upload_file(filename):
    return client.files.create(
        file=open(filename,'rb'),
        purpose='assistants')

df["Uploaded"] = df["File"].progress_apply(upload_file)

100%|██████████| 130/130 [00:13<00:00,  9.83it/s]


Поместим все загруженные файлы в векторную базу данных. Если файлы достаточно большие, то в процессе индексации можно применить **чанкование** - нарезку файлов на перекрывающиеся фрагменты с заданной длинной и перекрытием (в токенах). Мы зададим эти параметры, но поскольку в нашем случае текстовые файлы невелики - чанкование не потребуется.

In [29]:
vector_store = client.vector_stores.create(name='rag_store')

def add_to_store(file):
    client.vector_stores.files.create(
        vector_store_id=vector_store.id, 
        file_id=file.id,
        chunking_strategy={
            "type": "static",
            "static" : { "max_chunk_size_tokens" : 1000, "chunk_overlap_tokens" : 100 }
        }
        )

_ = df['Uploaded'].progress_apply(add_to_store)

100%|██████████| 130/130 [01:10<00:00,  1.84it/s]


Теперь мы можем в явном виде искать фрагменты в нашем векторном хранилище:

In [30]:
res = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="Какого цвета вино Зинфандель?",
    rewrite_query=True
)
for x in res.data:
    print(f"{len(x.content[0].text)} символов из файла {x.filename}, релевантность = {x.score}")

842 символов из файла Зинфандель.txt, релевантность = 0.9999999969414309
834 символов из файла Зинфандель.txt, релевантность = 0.48569737948984915
459 символов из файла Калифорния.txt, релевантность = 0.45973422471037917
876 символов из файла Зинфандель.txt, релевантность = 0.4486428687575345
263 символов из файла Орегон.txt, релевантность = 0.33300591614377084
2595 символов из файла Блауфранкиш.txt, релевантность = 0.1542271800239688
953 символов из файла Орегон.txt, релевантность = 0.11046288182563078
968 символов из файла Калифорния.txt, релевантность = 0.11046288182563078
1504 символов из файла Жюрансон.txt, релевантность = 0.09725001710844533
945 символов из файла Вионье.txt, релевантность = 0.09612696947896651



Теперь собираем собственно ассистента, который будет использовать RAG. Для этого определяем **инструмент** (tool) для поиска в нашем индексе, и указываем его при создании ассистента. Также важно задать хорошую инструкцию для ассистента (системный промпт): 

In [45]:
search_tool = {
    "type" : "file_search",
    "vector_store_ids" : [vector_store.id],
    "max_num_results" : 5,
    "include" : ["file_search_call.results"]
}

instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. С помощью инструмента поиска посмотри на всю имеющуюся
в твоем распоряжении информацию и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке. Обязательно используй либо MCP, либо инструмент поиска.
Если что-то непонятно, то лучше уточни информацию у пользователя.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
    tool_choice='required'
)

In [46]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)

К стейку лучше всего подойдут **полнотелые красные вина** с выраженной структурой и танинами, которые гармонируют с насыщенным вкусом мяса.

На основе анализа информации, вот лучшие варианты:

1. **Красное сухое вино из долины Маллеко (Чили)** — сочетается с запеченными овощами, свининой и **стейком на гриле**. Это вино обладает богатым вкусом, подходящим к жирному и сочному мясу.
2. **Темпранильо или Каберне Совиньон из Техаса (США)** — эти полнотелые красные вина идеально подходят к мясным блюдам, включая стейк.
3. **Нерелло Маскалезе с Сицилии (Италия)** — вулканическое вино с высокой кислотностью, танинами и нотами вишни, пряностей и трав. Отлично сбалансирует жирность стейка.

Рекомендую выбрать **сухое красное вино** с насыщенным вкусом — оно подчеркнёт сочность мяса и добавит блюду глубины.

Хотите, чтобы я подобрал конкретные вина из ассортимента магазина?

Посмотрим, из каких источников был получен этот ответ.

In [47]:
from openai.types.responses.response_output_message import ResponseOutputMessage

def find_obj(t,l):
    for x in l:
        if isinstance(x,t):
            return x
    return None

def print_citations(result):
    o = find_obj(ResponseOutputMessage,result.output)
    for x in o.content[0].annotations:
        if x.type == "file_citation":
            print(f"{x.filename}, idx={x.index}")

print_citations(res)

Марке.txt, idx=0
Техас.txt, idx=0
Сур.txt, idx=0
Свортленд.txt, idx=0
Сицилия.txt, idx=0


Мы видим, что ответ получился несколько однобоким, поскольку данные о сочетании вин и еды содержатся в текстовой базе знаний в разрозненном виде.

## Добавляем таблицу соответствий

Поскольку подбор блюда к вину является частой задачей, добавим к нашей базе знаний явную табличку соответствий блюд и вин, которая находится в файле `data/food_wine_table.md` в формате markdown.

In [48]:
with open("data/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
fw = "".join(food_wine)

tokenizer = get_tokenizer(model)
tokens = len(tokenizer.encode(fw))
print(f"Токенов: {tokens}, {len(fw)/tokens} chars/token")

Токенов: 15803, 2.6457001835094602 chars/token


In [49]:
printx(fw[:1000])

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.
Бефстроганов | Белые вина: выдержанные в дубе Шардоне, Пино Гриджо (лучше — из Северной Италии), Вердехо, Вермент

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [50]:
header = food_wine[:2]
header

['Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду\n',
 '--------|--------\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [51]:
import io

chunk_size = 600 * 5  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_chunks = []
i = 0
for x in tqdm(food_wine[2:]):
    s.append(x)
    if len("".join(s)) > chunk_size:
        f = client.files.create(
            purpose="assistants",
            file = (f'table_{i}.txt',io.BytesIO("".join(s).encode("utf-8")),'text/markdown')
        )
        client.vector_stores.files.create(file_id=f.id, vector_store_id=vector_store.id)
        uploaded_chunks.append(f)
        i+=1
        s = header.copy()
print(f"Uploaded {len(uploaded_chunks)} table chunks")

100%|██████████| 161/161 [00:11<00:00, 14.12it/s]

Uploaded 13 table chunks


Посмотрим, откуда теперь агент берёт информацию о соответствиях:

In [53]:
res = wine_agent("Какое вино подходит к стейку?")
printx(res.output_text)
print_citations(res)

К стейку подходят **сухие красные вина** с хорошей структурой и танинами, которые гармонируют с сочной текстурой мяса. Конкретный выбор зависит от типа стейка:

### 🔹 **К жирному стейку (Рибай, Т-бон)**
- **Каберне Совиньон** — мощное, с танинами, чёрными фруктами и дубовыми нотами. Идеально к прожарке **Medium** и **WellDone**.
- **Мальбек (Аргентина)** — шелковистое, с бархатистыми танинами, подходит к стейку на гриле.
- **Сира (Шираз)** — пряное, с перцем и тёмными ягодами, особенно хорошо к жареному мясу.
- **Выдержанные вина из Темпранильо (Рибера дель Дуэро)** — благородные, с кожей, табаком и древесными оттенками.

### 🔹 **К нежному стейку (Филе-миньон)**
- **Пино Нуар** — лёгкое, элегантное, с вишнёвыми и землистыми нотами. Не перебивает тонкий вкус мяса.
- **Нерелло Маскалезе (Сицилия)** — вулканическое вино с высокой кислотностью, подчёркивает сочность.
- **Выдержанное Мерло** — округлое, с мягкими танинами и фруктовым профилем.

### Также рекомендуются:
- **Бордо Правого берега** (Мерло + Каберне Фран)
- **Супертосканские вина**
- **Выдержанные ассамбляжи из Гренаша, Сиры и Мурведра (Рон, Приорат)**

Хотите, чтобы я подобрал **конкретные вина из ассортимента магазина** с ценами и возможностью добавить в корзину?

Марке.txt, idx=0
table_10.txt, idx=0
table_1.txt, idx=0
Франшхук.txt, idx=0
table_2.txt, idx=0


## Многоагентное тестирование

Когда мы сделали такого бота, возникает вопрос, как его тестировать. Для этого возможно несколько вариантов:

* Ручное тестирование (примерно то, что мы проделали выше)
* Автоматическое тестирование на заранее заготовленном датасете диалогов, с формализованной проверкой метрик. Такое тестирование удобно проводить с помощью специализированных фреймворков, например, RAGAS.

Мы в данном примере предлагаем промежуточный по сложности подход - это **многоагентное тестирование**, при котором мы инструктируем другого агента, чтобы он побеседовал с нашим винным ассистентом в определённом ключе. В итоге у нас могут получиться различные сценарии диалога, глядя на которые можно будет определить направление доработки исходного ассистента.

In [54]:
instruction = """
Ты - опытный сомелье, в задачу которого входит отвечать на вопросы пользователя про вина
и рекомендовать лучшие вина к еде. Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке. Говори не слишком длинными разговорными фразами.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[mcp_tool,search_tool,AddToCart,Handover],
)

In [55]:
instruction_user = """
Ты - простой человек, и тебе нужно выбрать вино в интернет-магазине для ужина.
Ты не очень много понимаешь в винах, и хочешь побольше распросить сомелье о разных вариантах.
Ты хотел бы съесть стейк, но готов изменить свой выбор еды, если поймёшь хорошее сочетание с вином. 
Говори простым языком, короткими разговорными фразами. Когда ты удовлетворём выбором, попроси 
найти самое недорогое вино данного типа, затем положи его в корзину. В конце попроси соединить тебя 
с оператором, чтобы оформить доставку. Каждое действие выполняй в одной фразой в ответ на сообщение сомелье. Никогда 
не продолжай диалог, больше, чем одной фразой. Не пиши реплики от лица сомелье или кого-то другого.
"""

user = Agent(instruction=instruction_user)

msg = "Добрый день! Поможете мне выбрать вино?"
handover = False
for i in range(10):
    printx(f"**Посетитель:** {msg}")
    msg = wine_agent(msg).output_text
    printx(f"**Сомелье:** {msg}")
    if handover:
        break
    msg = user(msg).output_text

**Посетитель:** Добрый день! Поможете мне выбрать вино?

**Сомелье:** Добрый день! Конечно, помогу с выбором вина. 😊  
Расскажите, пожалуйста:  

- К какому блюду подбираете вино? (например, стейк, рыба, сыр, десерт)  
- Предпочитаете **красное, белое или розовое** вино?  
- Какой вкус нравится — **сухое, полусухое, полусладкое или сладкое**?  
- Есть ли пожелания по стране или цене?  

Или просто скажите — на какое событие подбираете вино, и я сделаю отличную подборку!

**Посетитель:** Хочу красное сухое вино к стейку, но если есть классное сочетание с другим блюдом — могу и поменять ужин.

**Сомелье:** Отлично! Вот топ-5 **красных сухих вин**, идеально подходящих к стейку — все они мощные, выдержанные и отлично раскрываются с мясом:

1. **Сассикая (Италия)** — 27 799 ₽  
   *Легендарное супертосканское вино* на основе Каберне Совиньон. Глубокое, сложное, с шелковистыми танинами. Подойдёт к **стейку Рибай или Вагю** — это настоящий гастрономический дуэт.

2. **Сиепи Мазей (Италия)** — 15 999 ₽  
   *Концентрированное вино* из региона Монтепульчано. Богатый вкус с нотами табака, вишни и специй. Отлично сбалансирует жирность стейка.

3. **Палафрено (Италия)** — 14 999 ₽  
   Элегантное вино с гармоничной структурой. Хорошо подойдёт к **филе-миньону с грибным соусом** — можно даже немного поменять ужин в пользу чего-то более изысканного.

4. **Антино Тиньянелло (Италия)** — 14 499 ₽  
   Современное супертосканское вино с Каберне Совиньон, Сира и Петит Вердо. Сочное, с пряными и бальзамическими нотами — идеально к **стейку на гриле**.

5. **Шато Монроз (Франция)** — 12 999 ₽  
   *Бордо высокого класса* из Помероля. Очень элегантное, с тонами чёрной вишни, трюфеля и шоколада. Подойдёт к **стейку с трюфельным маслом** — если хотите устроить настоящий праздник вкуса.

### Что посоветую:
- Если хотите **максимально классическое сочетание** — берите **Шато Монроз** или **Антино Тиньянелло**.
- Если хотите **превратить ужин в событие** — **Сассикая + стейк Вагю** — это must try.

Хотите добавить одно из вин в корзину? Или подобрать что-то более бюджетное?

**Посетитель:** Давайте лучше подберём что-то подешевле, а то цены кусаются.

...

 + Обрабатываем: AddToCart ({"wine_name": "ДЖАСТ КБСВ (Франция)", "count": 1})


**Сомелье:** Отлично! Вино **ДЖАСТ КБСВ (Франция)** уже в вашей корзине.  
Это отличный выбор к стейку — насыщенное, с характером, и при этом очень выгодное.

Хотите посмотреть корзину или добавить что-то ещё? Например, бокалы или закуску?

**Посетитель:** Давайте лучше сразу перейдём к оформлению — соедините меня с оператором, пожалуйста.

 + Обрабатываем: Handover ({"reason": "Пользователь хочет перейти к оформлению заказа и связаться с оператором"})


**Сомелье:** Оператор уже в пути — он поможет вам с оформлением заказа и ответит на все вопросы.  
Спасибо, что доверились моим рекомендациям! 🍷

Хорошего вечера и приятного ужина со стейком и вином!

## Делаем ассистента проактивным

Мы обычно воспринимаем ассистентов как вопрос-ответных ботов, которые способны отвечать на вопросы пользователей. Однако с помощью системного промпта мы можем добавить им проактивности, чтобы они проявляли инициативу и сами инициировали диалог. Например:

In [64]:
instruction = """
Ты - опытный сомелье, продающий вино в магазине. Твоя задача - отвечать на вопросы пользователя
про вина, рекомендовать лучшие вина к еде, а также искать вина в прайс-листе нашего магазина,
а также проактивно предлагать пользователю приобрести вина, отвечающие его потребностям. В ответ
на сообщение /start поинтересуйся, что нужно пользователю, предложи ему какой-то
интересный вариант сочетания еды и вине, и попытайся продать ему вино.
Посмотри на всю имеющуюся в твоем распоряжении информацию
и выдай одну или несколько лучших рекомендаций.
Если вопрос касается конкретных вин или цены, то вызови функцию MCP-сервера Wine-Shop.
Для передачи управления оператору - вызови фукцию Handover. Для добавления вина в корзину
используй AddToCart. Для просмотра корзины: ShowCart. Все названия вин, цветов, кислотности
пиши на русском языке.
Если что-то непонятно, то лучше уточни информацию у пользователя. Общайся достаточно короткими 
разговорными фразами, не используй перечисления, списки, длинные выдержки текста.
"""

wine_agent = Agent(
    instruction=instruction,
    tools=[search_tool, mcp_tool, Handover, AddToCart, ShowCart],
)

In [57]:
printx(wine_agent('/start').output_text)

Добрый день! 😊  
Ищете вино к ужину или на какое-то событие?  

Кстати, отличное сочетание: **стейк на гриле + сухое красное вино** — например, Каберне Совиньон. Подчеркивает сочность мяса и создаёт настоящий гастрономический момент.  

Хочу предложить вам что-то подобное? Или, может, интересуете вино к чему-то другому — рыбе, сырам, десерту?  
Могу сразу подобрать лучшие варианты из ассортимента.

In [58]:
printx(wine_agent('Хочу недорогое красное вино под стейк').output_text)

Отлично! Вот лучшие **недорогие красные сухие вина** под стейк — всё вкусно, доступно и проверено:

1. **АВТОРСКОЕ №1 (Россия)** — **119 ₽**  
   Простое, но приятное. Хорошо подойдёт, если не хотите переплачивать.

2. **АДАГУМ КБСВ (Россия)** — **119 ₽**  
   На основе Каберне Совиньон. Ноты смородины и трав — классика к мясу.

3. **ДЖАСТ КБСВ (Франция)** — **149 ₽**  
   Французское вино с хорошей структурой. Лучшее соотношение цены и качества.

4. **ДЖАСТ Мерло (Франция)** — **149 ₽**  
   Мягкое, бархатистое. Хорошо к стейку средней прожарки.

5. **Тамани Каберне (Россия)** — **229 ₽**  
   Более насыщенное, с выдержкой. Чувствуются дуб, табак, тёмные ягоды.

### Рекомендую:  
**ДЖАСТ КБСВ из Франции за 149 ₽** — это идеальный баланс цены, качества и стиля. Отлично раскроется с вашим стейком.

Хотите добавить его в корзину?

## Делаем винного ассистента в телеграме

Для демонстрации возможностей бота мы реализуем такого ассистента в виде телеграм-бота. Конечно, для реализации полноценного телеграм-бота необходимо использовать виртуальную машину и режим webhooks, но в нашем случае мы ограничимся режимом поллинга, и запустим бота прямо в Datasphere.

> Прежде, чем запускать код ниже, необходимо создать чат-бота, пообщавшись с [@botfather](http://t.me/botfather), и разместить его секрет в виде секрета в Datasphere.

Для начала установим необходимую библиотеку:

In [59]:
%pip install --quiet telebot


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [70]:
import telebot

telegram_token = os.environ["tg_token"]

bot = telebot.TeleBot(telegram_token)

sessions = {}

# Обработчик команды /start
@bot.message_handler(commands=["start"])
def start(message):
    session_id = message.chat.id
    print(f"Starting on session {session_id}, msg={message.text}")
    ans = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, ans.output_text)


# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    session_id = message.chat.id
    print(f"Answering on session {session_id}, msg={message.text}")
    answer = wine_agent(message.text, session_id=session_id)
    bot.send_message(message.chat.id, answer.output_text)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
Answering on session 6483548294, msg=Привет!
Answering on session 6483548294, msg=Добавь в корзину окситоцин!
Answering on session 6483548294, msg=Да, шираз
Answering on session 6483548294, msg=Да
 + Обрабатываем: AddToCart ({"wine_name": "ЧОЛК ХИЛЛ ШИРАЗ", "count": 1})
Answering on session 6483548294, msg=Что в моей корзине
 + Обрабатываем: ShowCart ({})
Answering on session 6483548294, msg=Соедини с оператором, хочу доставку в сибирь!
 + Обрабатываем: Handover ({"reason": "Пользователь хочет оформить доставку в Сибирь, требуется уточнение адреса, сроков и стоимости"})


## Удаляем лишнее

В заключении удалим все созданные ресурсы. Для простоты мы удалим все ресурсы из текущего облака/проекта. 

**ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты на основе Responses API в облаке!

In [71]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)

 + Deleting vector store id=fvt2f4411sf9mqeevj34
 + Deleting file id=fvt2m4d922sctkk3eaku
 + Deleting file id=fvt7d5anbodkop8n9ke2
 + Deleting file id=fvts7r02770lusl2nli0
 + Deleting file id=fvtgv4scdfp067igjrv0
 + Deleting file id=fvtj6278rmj74u9fs0gs
 + Deleting file id=fvtiiupgaaj9crsieu9d
 + Deleting file id=fvtguj4qe87dgluvpdv3
 + Deleting file id=fvti2v76cmhrin6g45i7
 + Deleting file id=fvt04a0e9r0vb70a4il1
 + Deleting file id=fvtgkvs4qd6lc8f1pfv3
 + Deleting file id=fvteba755n0tgg59co40
 + Deleting file id=fvtj8rumufocskeobqpo
 + Deleting file id=fvt502262ipc3msfv4rt
 + Deleting file id=fvtoekfked5s0cvlfd2s
 + Deleting file id=fvtlseim7v7o4rde7p84
 + Deleting file id=fvtnhspbjg18mn3mksqq
 + Deleting file id=fvtne23blluar2uhqqjn
 + Deleting file id=fvtbh758hpbpnu4ojdcr
 + Deleting file id=fvtkbt05vah38mpplp34
 + Deleting file id=fvt6bp4l73irleb9clej
 + Deleting file id=fvt4hk6mbfgb0c26itsc
 + Deleting file id=fvtur7q2lnvmpsh3mkgu
 + Deleting file id=fvtfajg2vc6n5q6hbnp4
 + Delet

## Выводы

OpenAI Responses API - это удобный протокол для работы с языковыми моделями, поддерживающий сохранение истории переписки, вызов локальных функций и MCP, а также инструменты семантического поиска. На основе Responses API построены многие библиотеки, совместимые с OpenAI. Поддержка протокола Responses API в Yandex Cloud позволяет использовать соответствующие модели в облаке со множеством OpenAI-совместимых инструментов. 